<a href="https://colab.research.google.com/github/MJannik/hu_wirtschaftsinformatik_2021/blob/main/Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asymetric Costs in Regression Models: Markdown Method

This Cell imports important libraries like "pandas" or "numphy". It also asks for permission to connect to your GDrive. After running the cell, you have to click on the link and insert the code.

In [1]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder

from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


The next cell imports the car data into this notebook. **IMPORTANT:** Before running the next cell, go to the "Bachelorseminar Wirtschaftsinformatik" folder in GDrive and click "Add shortcut to Drive". This has to be done because Google Collab can only acces folders on YOUR Drive and not on a shared Drive.

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Bachelorseminar Wirtschaftsinformatik/semWI_usedCar.csv")
#df = df.sample(1000)

In [3]:
df

,id,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand
0,0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,audi
1,1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,audi
2,2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,audi
3,3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,audi
4,4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,audi
...,...,...,...,...,...,...,...,...,...,...,...
99182,15152,Eos,2012,5990,Manual,74000,Diesel,125,58.9,2.0,vw
99183,15153,Fox,2008,1799,Manual,88102,Petrol,145,46.3,1.2,vw
99184,15154,Fox,2009,1590,Manual,70000,Petrol,200,42.0,1.4,vw
99185,15155,Fox,2006,1250,Manual,82704,Petrol,150,46.3,1.2,vw


## Data Preperation
This cell uses the function "OrdinalEncoder" to transform the categorial variables like car brand into numbers.

In [4]:
enc = OrdinalEncoder()
enc.fit(df[['model', 'transmission', 'fuelType', 'engineSize', 'brand']])
df[['model', 'transmission', 'fuelType', 'engineSize', 'brand']] = enc.transform(df[['model', 'transmission', 'fuelType', 'engineSize', 'brand']])

This cell splits the data into training, validation and test set. For this, it uses the numphy function split.

In [5]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [6]:
train

,id,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand
31981,10532,60.0,2018,9679,1.0,14044,4.0,150,64.2,3.0,2.0
62387,4994,49.0,2017,6995,1.0,20179,4.0,150,68.9,2.0,5.0
30462,9013,110.0,2019,19500,1.0,15,0.0,145,65.7,12.0,2.0
21504,55,162.0,2018,19995,0.0,24568,0.0,145,31.7,12.0,2.0
26653,5204,77.0,2017,11891,1.0,11373,4.0,30,54.3,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
81004,10606,187.0,2017,10170,1.0,43457,4.0,200,42.2,6.0,7.0
34806,13357,56.0,2018,32000,0.0,11007,0.0,145,37.2,12.0,2.0
56292,12018,32.0,2008,3995,0.0,123000,0.0,265,42.2,13.0,4.0
77008,6610,172.0,2015,5750,1.0,20451,4.0,20,62.8,2.0,7.0


This cell creates two lists. One for the dependent and one for the independent variables of the training dataset. 

In [7]:
X_train = train[['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']]
y_train = train[['price']]

This cell does the same for the validation set.


In [8]:
X_validate = validate[['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']]
y_validate = validate[['price']]

This cell does the same for the test set.


In [9]:
X_test = test[['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']]
y_test = test[['price']]

## Estimate Model
This cell uses the statsmodels library to run an OLS regression on the training dataset. Below you can see the results of the regression.

In [10]:
model = sm.OLS(y_train, X_train.astype(float)).fit()
predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.911
Model:                            OLS   Adj. R-squared (uncentered):              0.911
Method:                 Least Squares   F-statistic:                          7.642e+04
Date:                Sun, 15 Aug 2021   Prob (F-statistic):                        0.00
Time:                        15:38:47   Log-Likelihood:                     -6.0007e+05
No. Observations:               59512   AIC:                                  1.200e+06
Df Residuals:                   59504   BIC:                                  1.200e+06
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
model            4.4948      0.459      9.783      0.000       3.594       5.395
year             5.0259      0.088     56.879      0.000       4.853       5.199
transmission   719.0322     23.831     30.173      0.000     672.324     765.740
mileage         -0.2209      0.001   -184.466      0.000      -0.223      -0.219
fuelType       251.3500     16.628     15.116      0.000     218.759     283.941
tax             -2.6051      0.444     -5.865      0.000      -3.476      -1.734
mpg            -21.7673      1.804    -12.063      0.000     -25.304     -18.231
engineSize    1341.6968      6.432    208.604      0.000    1329.090    1354.303
==============================================================================
Omnibus:                    37822.952   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1682784.096
Skew:                           2.454   Prob(JB):                         0.00
Kurtosis:                      28.584   Cond. No.                     3.14e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.14e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Apply Model
This cell applies the model on the validation dataset. (It predicts the price for every car in the validation set)

In [11]:
validate_apply = pd.DataFrame(columns = ['predicted_price', 'actual_price'])
validate_apply["predicted_price"] = model.predict(X_validate)
validate_apply["actual_price"] = y_validate

In [12]:
validate_apply

,predicted_price,actual_price
44099,22116.489148,9495
16256,40394.245856,66991
3583,13065.004313,14495
79911,11363.653819,11900
84745,12554.076448,20990
...,...,...
67078,16593.173168,11599
74397,11708.656695,10300
45823,19522.023262,11850
44830,37452.525297,23998


##LinLin Function
This cell defines the LinearLinear cost function. It checks if the residual is greater than 0. If yes, it multiplies the ratio of asymetry which represents the asymetric costs. 

In [13]:
def linlin(ratio, md, actual, prediction): 
    residual = ((prediction + md) - actual)
    
    if residual > 0: # Underprediction
        actualCosts = abs(residual * ratio)
    else:                   # Overprediction
        actualCosts = abs(residual)    
    return actualCosts

## Actual Costs Function
This cell defines the function that calculates the actual costs. It uses the linlin function from the previous cell and applies it on our dataset with the actual and predicted price. Finally it uses mean() to calculate the average misprediction over all residuals.

In [14]:

def aK(ratio, md):
    
    validate_apply["actualCosts"] = validate_apply.apply(lambda x: linlin(ratio, md, x["actual_price"], x["predicted_price"]),axis=1)
    
    actualCosts = validate_apply["actualCosts"].mean()
    
    return actualCosts

## Markdown Algorithm
This cell is the implementation of the markdownalgorithm suggested by Gaurav Bansal.

In [33]:
def find_md(ratio):
  md = 0
  p = 1
  s = 1

  # This part determines d which defines the "direction" of our markdown
  if aK(ratio, p) < aK(ratio, 0):
   d = 1
  else:
   d = -1

  md_prev = 0
  md_next = md + s * d * p

  while True:
   s = 1
   md = md_prev

   while True:  
      md_prev = md
      md = md + s * d * p
      s = s * 2
      md_next = md + s * d * p
      if aK(ratio, md_next) > aK(ratio, md):
        break
      #print("markdown is: " + str(md))
      #print("costs are: " + str(aK(ratio, md)))
   if s <= 2:
     break
  optimal_md = md
  #print("The optimal markdown value is: " + str(md))
  return optimal_md

In [37]:
md_list = []
ratio_list = []
results = pd.DataFrame(columns=['Ratio','Markdown', 'Average Misprediction Costs', 'Average Misprediction Costs with md = 0'])

for i in range(0, 11, 1):
  ratio = 1 + (i/10)
  md = find_md(ratio)
  AMC = aK(ratio, md)
  AMC_md0 = aK(ratio, 0)
  md_list.append(md)
  ratio_list.append(ratio)
  results = results.append({'Ratio': ratio, 'Markdown': md, 'Average Misprediction Costs':AMC, 'Average Misprediction Costs with md = 0':AMC_md0 }, ignore_index=True)
  print("ratio: " + str(ratio) + " markdown: " + str(md) + " the average misprediction costs are: " + str(AMC) + " The AMC for md = 0 are: " + str(AMC_md0))

ratio: 1.0 markdown: -493 the average misprediction costs are: 3998.606597189818 The AMC for md = 0 are: 4018.3175658583746
ratio: 1.1 markdown: -759 the average misprediction costs are: 4166.345080838971 The AMC for md = 0 are: 4218.388902253295
ratio: 1.2 markdown: -1005 the average misprediction costs are: 4321.957867348092 The AMC for md = 0 are: 4418.4602386481865
ratio: 1.3 markdown: -1237 the average misprediction costs are: 4466.742215777822 The AMC for md = 0 are: 4618.5315750430855
ratio: 1.4 markdown: -1445 the average misprediction costs are: 4601.779516323907 The AMC for md = 0 are: 4818.602911438032
ratio: 1.5 markdown: -1637 the average misprediction costs are: 4728.283815747048 The AMC for md = 0 are: 5018.674247832943
ratio: 1.6 markdown: -1822 the average misprediction costs are: 4847.659605713495 The AMC for md = 0 are: 5218.745584227864
ratio: 1.7 markdown: -1986 the average misprediction costs are: 4960.293495165443 The AMC for md = 0 are: 5418.81692062278
ratio: 1

## Test Part
Now we test if the markdown also decreases the costs for the test set.


In [21]:
test_apply = pd.DataFrame(columns = ['predicted_price', 'actual_price'])
test_apply["predicted_price"] = model.predict(X_test)
test_apply["actual_price"] = y_test

In [22]:
test_apply

,predicted_price,actual_price
42974,18062.439903,12000
70072,8784.530386,8369
41811,20073.266382,22995
51298,16740.994415,20200
50333,20794.907281,13795
...,...,...
6265,27820.892003,32950
54886,24857.190044,42798
76820,8177.451793,6390
860,13057.288507,20990


In [23]:
def aK_test(ratio, md):
    
    test_apply["actualCosts"] = test_apply.apply(lambda x: linlin(ratio, md, x["actual_price"], x["predicted_price"]),axis=1)
    
    actualCosts = test_apply["actualCosts"].mean()
    
    return actualCosts

In [42]:
results_testset = pd.DataFrame(columns=['Ratio','Markdown', 'Average Misprediction Costs', 'Average Misprediction Costs with md = 0'])

for r, m in zip(ratio_list, md_list):
  
  AMC_testset = aK_test(r, m)
  results_testset = results_testset.append({'Ratio': r, 'Markdown': m, 'Average Misprediction Costs':AMC_testset, 'Average Misprediction Costs with md = 0':aK_test(r, 0) }, ignore_index=True)
  print("ratio: " + str(r) + " markdown: " + str(m) + " the average misprediction costs is: " + str(AMC_testset) + " The AMC for md = 0 is: " + str(aK_test(r, 0)))

ratio: 1.0 markdown: -493 the average misprediction costs is: 4074.430604862563 The AMC for md = 0 is: 4095.8300414757196
ratio: 1.1 markdown: -759 the average misprediction costs is: 4246.029866844886 The AMC for md = 0 is: 4301.555111830049
ratio: 1.2 markdown: -1005 the average misprediction costs is: 4405.390689294911 The AMC for md = 0 is: 4507.280182184372
ratio: 1.3 markdown: -1237 the average misprediction costs is: 4553.6460356428615 The AMC for md = 0 is: 4713.005252538699
ratio: 1.4 markdown: -1445 the average misprediction costs is: 4692.020878891747 The AMC for md = 0 is: 4918.730322892995
ratio: 1.5 markdown: -1637 the average misprediction costs is: 4821.727494251072 The AMC for md = 0 is: 5124.455393247337
ratio: 1.6 markdown: -1822 the average misprediction costs is: 4943.8265964114435 The AMC for md = 0 is: 5330.1804636016495
ratio: 1.7 markdown: -1986 the average misprediction costs is: 5059.15609669538 The AMC for md = 0 is: 5535.905533955988
ratio: 1.8 markdown: -2

In [45]:
results_testset


,Ratio,Markdown,Average Misprediction Costs,Average Misprediction Costs with md = 0
0,1.0,-493.0,4074.430605,4095.830041
1,1.1,-759.0,4246.029867,4301.555112
2,1.2,-1005.0,4405.390689,4507.280182
3,1.3,-1237.0,4553.646036,4713.005253
4,1.4,-1445.0,4692.020879,4918.730323
5,1.5,-1637.0,4821.727494,5124.455393
6,1.6,-1822.0,4943.826596,5330.180464
7,1.7,-1986.0,5059.156097,5535.905534
8,1.8,-2147.0,5168.076788,5741.630604
9,1.9,-2299.0,5272.346980,5947.355675
